In [1]:
from patentdata.corpus import USPublications

path = '/media/SAMSUNG1/Patent_Downloads'
ds = USPublications(path)

In [2]:
records = ds.get_records(["G", "06"])

554570 records located.


In [4]:
import pickle
PIK = "G06records.data"

with open(PIK, "wb") as f:
    pickle.dump(records, f)

In [8]:
len(records)

554570

In [9]:
records[80:90]

[(1058, '2001/20010405.ZIP', '20010405/UTIL0000/US20010000192A1-20010405.ZIP'),
 (1131, '2001/20010412.ZIP', '20010412/UTIL0000/US20010000265A1-20010412.ZIP'),
 (1167, '2001/20010419.ZIP', '20010419/UTIL0000/US20010000301A1-20010419.ZIP'),
 (1180, '2001/20010419.ZIP', '20010419/UTIL0000/US20010000314A1-20010419.ZIP'),
 (1181, '2001/20010419.ZIP', '20010419/UTIL0000/US20010000315A1-20010419.ZIP'),
 (1220, '2001/20010419.ZIP', '20010419/UTIL0000/US20010000354A1-20010419.ZIP'),
 (1222, '2001/20010419.ZIP', '20010419/UTIL0000/US20010000356A1-20010419.ZIP'),
 (1223, '2001/20010419.ZIP', '20010419/UTIL0000/US20010000357A1-20010419.ZIP'),
 (1271, '2001/20010426.ZIP', '20010426/UTIL0000/US20010000405A1-20010426.ZIP'),
 (1272, '2001/20010426.ZIP', '20010426/UTIL0000/US20010000406A1-20010426.ZIP')]

In [4]:
doc_generator = ds.patentdoc_generator(["G", "06"], sample_size=10)

patent_samples = [doc for doc in doc_generator]

554570 records located.
10 records sampled.


/patentdata/patentdata/models/claim.py:30: UserWarning: Detected claim number
                    does not equal passed claim number.
  does not equal passed claim number."""


Above is slow even for 100 records.  Can we add any kind of caching layer? It's probably the tars that are slowing everything down.

We cannot store the patentdoc objects as a pickled list because we haven't got the _get_state_ and _set_state_ on my custom classes as per here: https://docs.python.org/3/library/pickle.html#handling-stateful-objects

In [5]:
len(patent_samples)

10

In [6]:
import string
string.printable

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

String printable has length 100. Use string.printable[:-2] to filter down to 98 characters

In [7]:
import string

def string2int(doc_as_string, filter_printable=True):
    """ Convert a string into a list of integers representing its characters.
    
        If filter_printable is true limit to 98 printable characters."""
    if filter_printable:
        ints = [ord(c) if c in string.printable[:-2] else ord(" ") for c in doc_as_string ]
    else:
        ints = [ord(c) for c in doc_as_string]
    return ints

We also need to normalise the character array so that it scales from 0 to 97. Out of printable characters get replaced with a space.  

But dictionary is over the corpus? Corpus has a mapping from indexes to char_integers.  Or just a direct mapping from characters to numbers.

Functions below generate a dictionary using printable characters. These have been added as patentdoc methods.

In [8]:
def string2printint(doc_as_string):
    """ Convert a string into a list of integers representing its printable characters."""
    char_map = {c:i for i, c in enumerate(string.printable[:-2])}
    return [char_map[c] if c in char_map.keys() else char_map[" "] for c in doc_as_string]

def printint2string(doc_as_ints):
    """ Reconstruct document string from list of integers."""
    char_map = {i:c for i, c in enumerate(string.printable[:-2])}
    return "".join([char_map[i] for i in doc_as_ints])

In [9]:
doc_as_int = patent_samples[0].string2printint()
print(doc_as_int[45:55])

TypeError: 'dict' object is not callable

In [13]:
patent_samples[0].text[0:30]

'A social network system with a'

Can use can below to plot a bar chart of character frequencies:  

In [7]:
from collections import Counter
a = ['a', 'a', 'a', 'a', 'b', 'b', 'c', 'c', 'c', 'd', 'e', 'e', 'e', 'e', 'e']
letter_counts = Counter(a)
for l in letter_counts.keys():
    print(l)

b
e
a
d
c


In [36]:
def char_stats(docs):
    """ Given a list of document strings converted to character integers,
    return statistics."""
    sum_counter = Counter()
    for doc in docs:
        sum_counter += Counter(doc)
    print("Documents contain {0} unique characters.".format(len(sum_counter)))
    return sum_counter

In [37]:
docs_as_chars = [[c for c in pd.text] for pd in patent_samples]

In [35]:
c = char_stats(docs_as_int)

Documents contain 186 unique characters.


In [38]:
c_chars = char_stats(docs_as_chars)

Documents contain 186 unique characters.


In [39]:
c_chars

Counter({'\n': 49267,
         ' ': 1144736,
         '!': 15,
         '#': 35,
         '$': 67,
         '%': 231,
         '&': 60,
         "'": 1060,
         '(': 12416,
         ')': 12798,
         '*': 110,
         '+': 381,
         ',': 62670,
         '-': 11930,
         '.': 54987,
         '/': 4008,
         '0': 32798,
         '1': 34428,
         '2': 22456,
         '3': 11842,
         '4': 12405,
         '5': 8625,
         '6': 8909,
         '7': 5343,
         '8': 6319,
         '9': 3659,
         ':': 2898,
         ';': 4029,
         '<': 894,
         '=': 645,
         '>': 990,
         '?': 54,
         '@': 2,
         'A': 13610,
         'B': 4417,
         'C': 7253,
         'D': 6946,
         'E': 4314,
         'F': 11432,
         'G': 7134,
         'H': 2428,
         'I': 17893,
         'J': 492,
         'K': 1126,
         'L': 4532,
         'M': 6626,
         'N': 3481,
         'O': 3732,
         'P': 6189,
         'Q': 398,
   

In [53]:
docs_as_int = [string2int(pd.text) for pd in patent_samples]
docs_as_int_unfiltered = [string2int(pd.text, False) for pd in patent_samples]

In [57]:
stat_filtered = char_stats(docs_as_int)
stat_unfiltered = char_stats(docs_as_int_unfiltered)

Documents contain 94 unique characters.
Documents contain 186 unique characters.


In [58]:
stat_filtered

Counter({10: 49267,
         32: 1153151,
         33: 15,
         35: 35,
         36: 67,
         37: 231,
         38: 60,
         39: 1060,
         40: 12416,
         41: 12798,
         42: 110,
         43: 381,
         44: 62670,
         45: 11930,
         46: 54987,
         47: 4008,
         48: 32798,
         49: 34428,
         50: 22456,
         51: 11842,
         52: 12405,
         53: 8625,
         54: 8909,
         55: 5343,
         56: 6319,
         57: 3659,
         58: 2898,
         59: 4029,
         60: 894,
         61: 645,
         62: 990,
         63: 54,
         64: 2,
         65: 13610,
         66: 4417,
         67: 7253,
         68: 6946,
         69: 4314,
         70: 11432,
         71: 7134,
         72: 2428,
         73: 17893,
         74: 492,
         75: 1126,
         76: 4532,
         77: 6626,
         78: 3481,
         79: 3732,
         80: 6189,
         81: 398,
         82: 5231,
         83: 10021,
         84: 17

This is newline and 

In [65]:
# Working on processing characters using a dictionary built from printable characters (minus last two entries).
printable_chars = string.printable[:-2]
print([(i, c) for i, c in enumerate(printable_chars)])
char_map = {c:i for i, c in enumerate(printable_chars)}
print(char_map)

[(0, '0'), (1, '1'), (2, '2'), (3, '3'), (4, '4'), (5, '5'), (6, '6'), (7, '7'), (8, '8'), (9, '9'), (10, 'a'), (11, 'b'), (12, 'c'), (13, 'd'), (14, 'e'), (15, 'f'), (16, 'g'), (17, 'h'), (18, 'i'), (19, 'j'), (20, 'k'), (21, 'l'), (22, 'm'), (23, 'n'), (24, 'o'), (25, 'p'), (26, 'q'), (27, 'r'), (28, 's'), (29, 't'), (30, 'u'), (31, 'v'), (32, 'w'), (33, 'x'), (34, 'y'), (35, 'z'), (36, 'A'), (37, 'B'), (38, 'C'), (39, 'D'), (40, 'E'), (41, 'F'), (42, 'G'), (43, 'H'), (44, 'I'), (45, 'J'), (46, 'K'), (47, 'L'), (48, 'M'), (49, 'N'), (50, 'O'), (51, 'P'), (52, 'Q'), (53, 'R'), (54, 'S'), (55, 'T'), (56, 'U'), (57, 'V'), (58, 'W'), (59, 'X'), (60, 'Y'), (61, 'Z'), (62, '!'), (63, '"'), (64, '#'), (65, '$'), (66, '%'), (67, '&'), (68, "'"), (69, '('), (70, ')'), (71, '*'), (72, '+'), (73, ','), (74, '-'), (75, '.'), (76, '/'), (77, ':'), (78, ';'), (79, '<'), (80, '='), (81, '>'), (82, '?'), (83, '@'), (84, '['), (85, '\\'), (86, ']'), (87, '^'), (88, '_'), (89, '`'), (90, '{'), (91, '|

In [72]:
doc_3_as_ints = string2printint(patent_samples[3].text)

In [73]:
Counter(doc_3_as_ints)

Counter({0: 133,
         1: 118,
         2: 116,
         3: 60,
         4: 94,
         5: 85,
         6: 78,
         7: 94,
         8: 17,
         9: 9,
         10: 1875,
         11: 549,
         12: 996,
         13: 1312,
         14: 3144,
         15: 732,
         16: 472,
         17: 1064,
         18: 2362,
         19: 4,
         20: 446,
         21: 1103,
         22: 689,
         23: 1810,
         24: 2126,
         25: 491,
         26: 22,
         27: 1944,
         28: 2137,
         29: 2702,
         30: 562,
         31: 207,
         32: 338,
         33: 21,
         34: 430,
         35: 32,
         36: 68,
         37: 6,
         38: 16,
         39: 30,
         40: 23,
         41: 47,
         42: 38,
         43: 16,
         44: 88,
         45: 1,
         46: 2,
         47: 11,
         48: 24,
         49: 17,
         50: 18,
         51: 10,
         53: 28,
         54: 39,
         55: 83,
         56: 3,
         57: 7,
         58:

In [74]:
printint2string(doc_3_as_ints)

"A method for synchronizing copies of mirrored files between a first disk subsystem and a second disk subsystem, wherein said method comprises: \nA) writing a first block of data in the first disk subsystem; \nB) writing the first block of data in a log file in the first disk subsystem; \nC) reading the first block of data from the log file in the first disk subsystem and transmitting the first block of data to the second disk subsystem; and \nD) writing the first block of data in the second disk subsystem after receiving the first block of data from the first disk subsystem.\nThe method in claim 1 which further comprises: \nE) acknowledging writing the first block of data to a host system after writing the first block of data to the log file but before writing the first block of data in the second disk subsystem.\nThe method in claim 1 which further comprises: \nE) writing a second block of data in the first disk subsystem before the first block of data is read from the log file; \nF)

Why are the claims being shown first? PatentDoc description is the claims and vice versa. Must be a switch error somewhere.

Also we need claim numbers on claim text.

In [75]:
"\n\n".join(["1", "2"])

'1\n\n2'

In [77]:
patent_samples[3].claimset.text

"0001 The present invention generally relates to high availability disk subsystems, and more specifically to mirroring data in such a high availability disk subsystem. \n0002 Mirroring is the duplication of data on two or more disk drives. It is done primarily for data integrity protection. Originally, files were duplicated. This feature was available in the early 1970's on the Burroughs (now Unisys) 6700 series of computers. Later, entire disk drives were mirrored. Each disk write is duplicated to two or more disks. Reads from the disk(s) can be from either disk. This provided performance benefits in read operations and a small decrease of performances in write operations. Then, if there is a failure of one of the disks, reading can continue uninterrupted from the other disk. This significantly increases the data integrity and availability, and has an important ingredient of on-line transaction systems for a number of years. Availability was further enhanced by utilizing different dis

In [2]:
from nltk import sent_tokenize

In [78]:
from nltk.tokenize import sent_tokenize

In [81]:
sentence_list = sent_tokenize(patent_samples[3].claimset.text)

In [82]:
len(sentence_list)

222

In [83]:
sentence_list[33:44]

['However, mirroring is becoming much more attractive due to its simpler design as the cost per megabyte of disk space continues to drop year after year.',
 'It is also becoming used in RAID level 1/0 that provides mirroring in conjunction with striping for very high reliability/high performance applications.',
 '0008 Mirroring has other benefits too.',
 'It can be used as a tool to facilitate copies of different states of a database.',
 'This is accomplished by creating a mirror of a database.',
 'After the mirror is operational, it is broken at a particular moment.',
 'In this way, the secondary copy is the image of the database at a defined instant in time.',
 'A third copy of the database can be created at a different instant in time, etc.',
 '0009 Mirroring can also be utilized as a tool to facilitate backup on tapes.',
 'This is accomplished by creating a mirror of a disk or database.',
 'When the mirror is operational, it is broken at a specified moment in time.']

May need to clean paragraph numbers from text used for sentence segmentation.

In [84]:
sentence_list[33].split()

['However,',
 'mirroring',
 'is',
 'becoming',
 'much',
 'more',
 'attractive',
 'due',
 'to',
 'its',
 'simpler',
 'design',
 'as',
 'the',
 'cost',
 'per',
 'megabyte',
 'of',
 'disk',
 'space',
 'continues',
 'to',
 'drop',
 'year',
 'after',
 'year.']

In [94]:
patent_samples[3].claimset.paragraphs[1].set_words()

['0002',
 'Mirroring',
 'is',
 'the',
 'duplication',
 'of',
 'data',
 'on',
 'two',
 'or',
 'more',
 'disk',
 'drives',
 '.',
 'It',
 'is',
 'done',
 'primarily',
 'for',
 'data',
 'integrity',
 'protection',
 '.',
 'Originally',
 ',',
 'files',
 'were',
 'duplicated',
 '.',
 'This',
 'feature',
 'was',
 'available',
 'in',
 'the',
 'early',
 '1970',
 "'s",
 'on',
 'the',
 'Burroughs',
 '(',
 'now',
 'Unisys',
 ')',
 '6700',
 'series',
 'of',
 'computers',
 '.',
 'Later',
 ',',
 'entire',
 'disk',
 'drives',
 'were',
 'mirrored',
 '.',
 'Each',
 'disk',
 'write',
 'is',
 'duplicated',
 'to',
 'two',
 'or',
 'more',
 'disks',
 '.',
 'Reads',
 'from',
 'the',
 'disk',
 '(',
 's',
 ')',
 'can',
 'be',
 'from',
 'either',
 'disk',
 '.',
 'This',
 'provided',
 'performance',
 'benefits',
 'in',
 'read',
 'operations',
 'and',
 'a',
 'small',
 'decrease',
 'of',
 'performances',
 'in',
 'write',
 'operations',
 '.',
 'Then',
 ',',
 'if',
 'there',
 'is',
 'a',
 'failure',
 'of',
 'one',
 'o

In [95]:
from nltk import word_tokenize
word_tokenize(sentence_list[33])

['However',
 ',',
 'mirroring',
 'is',
 'becoming',
 'much',
 'more',
 'attractive',
 'due',
 'to',
 'its',
 'simpler',
 'design',
 'as',
 'the',
 'cost',
 'per',
 'megabyte',
 'of',
 'disk',
 'space',
 'continues',
 'to',
 'drop',
 'year',
 'after',
 'year',
 '.']

Could use methods to break paragraph into sentences and return either as a list of sentences or a list of word lists representing the sentence. Then we can look at sentence lengths across a corpus.

---
## Useful Statistics

Across a corpus (evaluate lazily):

- Unique number of characters (Counter set);
- Unique number of tokens (vocabulary);
- Total number of documents
- Total number of paragraphs
- Average number of paragraphs
- Histogram distribution of number of paragraphs per document
- Total number of sentences
- Average number of sentences per document
- Histogram distribution of number of sentences per document
- Histogram distribution of number of sentences per paragraph
- Average number of sentences per paragraph
- Total number of tokens
- Average number of tokens per document
- Histogram distribution of tokens per document
- Token counts across corpus
- Histogram distribution of claims per document
- Length of total characters in document (sequence length)
- Length of total tokens in document (sequence length)

Compare statistics for publications and grant.

Save after every 1000.

Start with per document statistics. These can be at the patentdoc level. These can include:
- Counter object for unfiltered characters.
- Counter object for filtered characters.
- Counter object for unfiltered tokens.
- Paragraph count.
- Sentence count.
- Average sentence per paragraph (Paragraph Count / Sentence Count).

In [6]:
pd1 = patent_samples[0]

In [9]:
pd1.description.paragraph_count

87

In [10]:
pd1.description.get_paragraph(2).sentences

RecursionError: maximum recursion depth exceeded in comparison

In [15]:
% timeit doc = ds.search_files("US20140068740A1")

The slowest run took 23.27 times longer than the fastest. This could mean that an intermediate result is being cached.
1.79 ms ± 1.4 ms per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [16]:
doc

NameError: name 'doc' is not defined